**Troubleshooting Scripts - Plan Guides**

Dmitri V. Korotkevitch (MCM, MVP)

email: [dk@aboutsqlserver.com](mailto:dk@aboutsqlserver.com)      blog: [https://aboutsqlserver.com](https://aboutsqlserver.com/) code: [https://github.com/aboutsqlserver/code](https://github.com/aboutsqlserver/code)

SQL Server Advanced Troubleshooting and Performance Tuning (O'Reilly, 2022)      ISBN: 978-1098101923

**Check Existing Plan Guides**

If message column indicates that plan guide is not valid

In [ ]:
IF OBJECT_ID(N'tempdb..#PG') IS NOT NULL 
	DROP TABLE #PG;
GO

CREATE TABLE #PG
(
	database_id SMALLINT NOT NULL,
	[database] SYSNAME NOT NULL,
	plan_guide_id INT NOT NULL,
	name SYSNAME NOT NULL,
	scope_type_desc NVARCHAR(60) NULL,
	is_disabled BIT NOT NULL,
	[message] NVARCHAR(1024) NULL
);

EXEC master..sp_MSforeachdb 
N'USE[?];
INSERT INTO #PG(database_id, [database], plan_guide_id, name, scope_type_desc, is_disabled, [message] )
    SELECT DB_ID(''?''), DB_NAME(), pg.plan_guide_id, pg.name, pg.scope_type_desc, pg.is_disabled, vpg.[message]
	FROM 
		sys.plan_guides pg WITH (NOLOCK)
			OUTER APPLY 
			(
				SELECT [message]
				FROM sys.fn_validate_plan_guide(pg.plan_guide_id)
			) vpg';

SELECT *
FROM #PG
ORDER BY [database_id], [name];

**Manage Plan Guides**

@operations - DROP, DROP ALL, ENABLE, ENABLE ALL, DISABLE, DISABLE ALL

@name - plan guide name

In [ ]:
EXEC sp_control_plan_guide 
	@operation = N'DROP'
	,@name = N'<name>';

**Enable FORCE parameterization**

Replace statement and name as needed

In [ ]:
DECLARE
	@stmt NVARCHAR(MAX)
	,@params NVARCHAR(MAX)
	,@query NVARCHAR(MAX) = N'statement';

EXEC sp_get_query_template
	@querytext = @query
	,@templatetext = @stmt OUTPUT
	,@params = @params OUTPUT;

EXEC sp_create_plan_guide
	@type = N'TEMPLATE'
	,@name = N'forced_parameterization_plan_guide'
	,@stmt = @stmt
	,@module_or_batch = NULL
	,@params = @params
	,@hints = N'OPTION (PARAMETERIZATION FORCED)';
GO

**Enable SIMPLE parameterization**

Step 1 - get the statement and parameters from plan cache. Use correct filter in WHERE clause to find the statement

In [ ]:
SELECT
	SUBSTRING(qt.text, (qs.statement_start_offset/2)+1,
	((
		CASE qs.statement_end_offset
			WHEN -1 THEN DATALENGTH(qt.text)
			ELSE qs.statement_end_offset
		END - qs.statement_start_offset)/2)+1) AS SQL
	,qt.text AS [Full SQL]
FROM
	sys.dm_exec_query_stats qs WITH (NOLOCK)
		CROSS APPLY sys.dm_exec_sql_text(qs.sql_handle) qt
WHERE
	qt.text LIKE '%filter%'
OPTION(RECOMPILE, MAXDOP 1);

Step 2 - Create the plan guide. Replace @stmt and @params based on results from Step 1

In [ ]:
DECLARE
	@stmt NVARCHAR(MAX) = N'select OrderId from dbo . Orders where Fulfilled = @0'
	,@params NVARCHAR(MAX) = N'@0 int'

EXEC sp_create_plan_guide
	@type = N'TEMPLATE'
	,@name = N'simple_parameterization_plan_guide'
	,@stmt = @stmt
	,@module_or_batch = NULL
	,@params = @params
	,@hints = N'OPTION (PARAMETERIZATION SIMPLE)';